<a href="https://colab.research.google.com/github/drojassa/Poincare/blob/main/coors_poincarre_theta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Required libraries for the transcripted code
import pandas as pd
import numpy as np
from numpy.linalg import eig
import matplotlib.pyplot as plt
from math import pi, sin, cos, atan, asin
import cmath
import ipywidgets as widgets
from IPython.display import display
from matplotlib.widgets import Slider



# Define sliders for alpha, delta, Coef11, and Coef22
alpha_slider = widgets.FloatSlider(min=(45-10), max=(45+10), step=0.01, value=45, description='alpha')
delta_slider = widgets.FloatSlider(min=(12.8-10), max=(12.8+10), step=0.01, value=12.8, description='delta')
Coef11_slider = widgets.FloatSlider(min=0.5, max=1, step=0.001, value=0.935, description='Coef11')
Coef22_slider = widgets.FloatSlider(min=0.5, max=1, step=0.001, value=0.930, description='Coef22')


# Read Excel data
data = pd.read_excel("coord_pola_theta.xlsx")

# Angles from the sheet
theta_exp = data.iloc[:, 2] * pi / 180
theta1_mode1 = (data.iloc[:, 6] - 331.5) * pi / 180
theta2_mode1 = (data.iloc[:, 7] - 44) * pi / 180
theta1_mode2 = (data.iloc[:, 8] - 331.5) * pi / 180
theta2_mode2 = (data.iloc[:, 9] - 44) * pi / 180

# Pair of angles for each mode
xi_mode1 = theta1_mode1
chi_mode1 = xi_mode1 - theta2_mode1 - pi / 2
xi_mode2 = theta1_mode2
chi_mode2 = xi_mode2 - theta2_mode2 - pi / 2

# Constants
n1 = 1
n2 = 1.5
#delta = 18 * pi / 180
theta = np.arange(0, pi/2, 0.005)
deltaxi = 5 * pi / 180
deltachi = 5 * pi / 180

# Initialize arrays
num_xi_mode1 = len(xi_mode1)
num_xi_mode2 = len(xi_mode2)

X1 = np.zeros(num_xi_mode1)
Y1 = np.zeros(num_xi_mode1)
Z1 = np.zeros(num_xi_mode1)
error1 = np.zeros((num_xi_mode1, 3))

X2 = np.zeros(num_xi_mode2)
Y2 = np.zeros(num_xi_mode2)
Z2 = np.zeros(num_xi_mode2)
error2 = np.zeros((num_xi_mode2, 3))

# Loop over xi_mode1 and xi_mode2 to calculate coordinates and errors
for j in range(num_xi_mode1):
    X1[j] = cos(2 * xi_mode1[j]) * cos(2 * chi_mode1[j])
    error1[j, 0] = abs(2 * sin(2 * xi_mode1[j]) * cos(2 * chi_mode1[j]) * deltaxi) +abs(2 * cos(2 * xi_mode1[j]) * sin(2 * chi_mode1[j]) * deltachi)
    Y1[j] = sin(2 * xi_mode1[j]) * cos(2 * chi_mode1[j])
    error1[j, 1] = abs(2 * cos(2 * xi_mode1[j]) * cos(2 * chi_mode1[j]) * deltaxi) +abs(2 * sin(2 * xi_mode1[j]) * sin(2 * chi_mode1[j]) * deltachi)
    Z1[j] = -sin(2 * chi_mode1[j])
    error1[j, 2] = abs(2 * cos(2 * chi_mode1[j]) * deltachi)

for j in range(num_xi_mode2):
    X2[j] = cos(2 * xi_mode2[j]) * cos(2 * chi_mode2[j])
    error2[j, 0] = abs(2 * sin(2 * xi_mode2[j]) * cos(2 * chi_mode2[j]) * deltaxi) +abs( 2 * cos(2 * xi_mode2[j]) * sin(2 * chi_mode2[j]) * deltachi)
    Y2[j] = sin(2 * xi_mode2[j]) * cos(2 * chi_mode2[j])
    error2[j, 1] = abs(2 * cos(2 * xi_mode2[j]) * cos(2 * chi_mode2[j]) * deltaxi) + abs(2 * sin(2 * xi_mode2[j]) * sin(2 * chi_mode2[j]) * deltachi)
    Z2[j] = -sin(2 * chi_mode2[j])
    error2[j, 2] = abs(2 * cos(2 * chi_mode2[j]) * deltachi)

# Initialize arrays for plotting
x1 = np.zeros(len(theta))
y1 = np.zeros(len(theta))
z1 = np.zeros(len(theta))
x2 = np.zeros(len(theta))
y2 = np.zeros(len(theta))
z2 = np.zeros(len(theta))


#Coef11=0.935
#Coef22=0.930

# Loop over theta


def update_plot(alpha, delta, Coef11, Coef22):
  alpha*=pi/180
  delta*=pi/180
  for k in range(len(theta)):
      theta2 = asin((n1 / n2) * sin(theta[k]))

      t1s = (2 * n1 * cos(theta[k])) / (n1 * cos(theta[k]) + n2 * cos(theta2))
      t2s = (2 * n2 * cos(theta2)) / (n1 * cos(theta[k]) + n2 * cos(theta2))
      t1p = (2 * n1 * cos(theta[k])) / (n2 * cos(theta[k]) + n1 * cos(theta2))
      t2p = (2 * n2 * cos(theta2)) / (n2 * cos(theta[k]) + n1 * cos(theta2))

      rs = -t1s * t2s * t1s * t2s
      rp = t1p * t2p * t1p * t2p

      # Mirror M2
      M2 = np.array([[rp, 0], [0, rs]])

      ts = t1s * t2s
      tp = t1p * t2p
      Mout = np.array([[ts, 0], [0, tp]])
      #alpha = 45 * pi / 180  # Angle from the x'y' to xy base

      # Mirror M1
      r11 = -cmath.exp(1j * (delta / 2))
      r12 = cmath.exp(-1j * (delta / 2))
      M1 = np.array([[np.sqrt(Coef11) * r11, 0], [0, np.sqrt(Coef22) * r12]])

      T = np.array([[cos(alpha), sin(alpha)], [-sin(alpha), cos(alpha)]])

      # Round-trip matrix
      J_RT = np.dot(np.dot(T, M1), np.dot(T, M2))

      # Eigenvalues and eigenvectors
      D, V = eig(J_RT)

      # The two eigenvectors
      V1 = np.dot(Mout, V[:, 0])
      V2 = np.dot(Mout, V[:, 1])

      # Parameters of the polarization ellipse of the two modes
      phi_V1 = cmath.phase(V1[1] / V1[0])
      phi_V2 = cmath.phase(V2[1] / V2[0])
      epsilon_V1 = atan(abs(V1[1]) / abs(V1[0]))
      epsilon_V2 = atan(abs(V2[1]) / abs(V2[0]))

      # Calculate the coordinates of the modes on the Poincaré sphere
      x1[k] = cos(2 * epsilon_V1)
      y1[k] = sin(2 * epsilon_V1) * cos(phi_V1)
      z1[k] = sin(2 * epsilon_V1) * sin(phi_V1)
      x2[k] = cos(2 * epsilon_V2)
      y2[k] = sin(2 * epsilon_V2) * cos(phi_V2)
      z2[k] = sin(2 * epsilon_V2) * sin(phi_V2)

  # Plotting the results
  fig, axs = plt.subplots(3)
  # X coordinates
  axs[0].errorbar(theta_exp*180/pi, X1, error1[:, 0], fmt='r*', label='mode 1')
  axs[0].errorbar(theta_exp*180/pi, X2, error2[:, 0], fmt='g*', label='mode 2')
  axs[0].plot(theta*180/pi, x1, 'r-')
  axs[0].plot(theta*180/pi, x2, 'g-')
  axs[0].grid(True)
  axs[0].set(ylabel='X')

  # Y coordinates
  axs[1].errorbar(theta_exp*180/pi, Y1, error1[:, 1], fmt='r*', label='mode 1')
  axs[1].errorbar(theta_exp*180/pi, Y2, error2[:, 1], fmt='g*', label='mode 2')
  axs[1].plot(theta*180/pi, y1, 'r-')
  axs[1].plot(theta*180/pi, y2, 'g-')
  axs[1].grid(True)
  axs[1].set(ylabel='Y')

  # Z coordinates
  axs[2].errorbar(theta_exp*180/pi, Z1, error1[:, 2], fmt='r*', label='mode 1')
  axs[2].errorbar(theta_exp*180/pi, Z2, error2[:, 2], fmt='g*', label='mode 2')
  axs[2].plot(theta*180/pi, z1, 'r-')
  axs[2].plot(theta*180/pi, z2, 'g-')
  axs[2].grid(True)
  axs[2].set(xlabel='theta', ylabel='Z')

  plt.show()
widgets.interact(update_plot, alpha=alpha_slider, delta=delta_slider, Coef11=Coef11_slider, Coef22=Coef22_slider)

interactive(children=(FloatSlider(value=45.0, description='alpha', max=55.0, min=35.0, step=0.01), FloatSlider…

<function __main__.update_plot(alpha, delta, Coef11, Coef22)>